<a href="https://colab.research.google.com/github/Kacper-W-Kozdon/notebook-testing-ivy/blob/main/Sarcasm_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files
from google.colab import userdata
import os

In [3]:
files.upload();

Saving kaggle.json to kaggle.json


In [4]:
!mkdir ~/.kaggle

In [5]:
!cp kaggle.json ~/.kaggle/

In [6]:
!chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets list

ref                                                   title                                           size  lastUpdated          downloadCount  voteCount  usabilityRating  
----------------------------------------------------  ---------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
syedanwarafridi/vehicle-sales-data                    Vehicle Sales Data                              19MB  2024-02-21 20:16:17          16108        287  1.0              
sudarshan24byte/online-food-dataset                   Online Food Dataset                              3KB  2024-03-02 18:50:30           5864        108  0.9411765        
nbroad/gemma-rewrite-nbroad                           gemma-rewrite-nbroad                             8MB  2024-03-03 04:52:39            761         69  1.0              
arnavvvvv/spotify-music                               Top Spotify Songs                               47KB  2024-03-06 05:20:29        

In [8]:
!git clone https://github.com/Kacper-W-Kozdon/demos.git
!git clone https://github.com/Kacper-W-Kozdon/ivy.git

Cloning into 'demos'...
remote: Enumerating objects: 3152, done.
remote: Counting objects: 100% (608/608), done.
remote: Compressing objects: 100% (219/219), done.
remote: Total 3152 (delta 447), reused 438 (delta 358), pack-reused 2544
Receiving objects: 100% (3152/3152), 16.15 MiB | 17.61 MiB/s, done.
Resolving deltas: 100% (1820/1820), done.
Cloning into 'ivy'...
remote: Enumerating objects: 188649, done.
remote: Counting objects: 100% (447/447), done.
remote: Compressing objects: 100% (180/180), done.
remote: Total 188649 (delta 300), reused 352 (delta 267), pack-reused 188202
Receiving objects: 100% (188649/188649), 177.18 MiB | 18.61 MiB/s, done.
Resolving deltas: 100% (151521/151521), done.


In [9]:
!pip install -U -q ivy mlflow datasets>=2.14.5 nlp 2>/dev/null
import ivy

In [26]:
!kaggle datasets download -d danofer/sarcasm
!cp -f sarcasm.zip '/content/demos/Contributor_demos/Sarcasm Detection/'
!unzip '/content/demos/Contributor_demos/Sarcasm Detection/sarcasm.zip' -d '/content/demos/Contributor_demos/Sarcasm Detection/'

sarcasm.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/demos/Contributor_demos/Sarcasm Detection/sarcasm.zip
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/test-balanced.csv  
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/test-unbalanced.csv  
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/train-balanced-sarc.csv.gz  
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/train-balanced-sarcasm.csv  


In [23]:
# Import necessary libraries
import pandas as pd  # For data manipulation and analysis
import gc  # For garbage collection to manage memory
import re  # For regular expressions
import numpy as np  # For numerical operations and arrays

import warnings  # For handling warnings
warnings.filterwarnings("ignore")  # Ignore warning messages

import torch  # PyTorch library for deep learning
from transformers import AutoModel, AutoTokenizer  # Transformers library for natural language processing
from transformers import TextDataset, LineByLineTextDataset, DataCollatorForLanguageModeling, \
pipeline, Trainer, TrainingArguments, DataCollatorWithPadding  # Transformers components for text processing
from transformers import AutoModelForSequenceClassification  # Transformer model for sequence classification

from nlp import Dataset  # Import custom 'Dataset' class for natural language processing tasks
from imblearn.over_sampling import RandomOverSampler  # For oversampling to handle class imbalance
import datasets  # Import datasets library
from datasets import Dataset, Image, ClassLabel  # Import custom 'Dataset', 'ClassLabel', and 'Image' classes
from transformers import pipeline  # Transformers library for pipelines
from bs4 import BeautifulSoup  # For parsing HTML content

import matplotlib.pyplot as plt  # For data visualization
import itertools  # For working with iterators
from sklearn.metrics import (  # Import various metrics from scikit-learn
    accuracy_score,  # For calculating accuracy
    roc_auc_score,  # For ROC AUC score
    confusion_matrix,  # For confusion matrix
    classification_report,  # For classification report
    f1_score  # For F1 score
)

from datasets import load_metric  # Import load_metric function to load evaluation metrics

from tqdm import tqdm  # For displaying progress bars
tqdm.pandas()  # Enable progress bars for pandas operations

In [50]:
df = pd.read_csv("/content/demos/Contributor_demos/Sarcasm Detection/train-balanced-sarcasm.csv")
df = df.drop_duplicates()

In [51]:
df = df[['comment', 'label']]
df = df[~df['comment'].isnull()]
df = df[~df['label'].isnull()]
print(df.sample(15))

                                                  comment  label
209332   Sexists aren't going to vote for Clinton anyway.      0
194979    Maybe the actual OP did murder his whole family      0
350518                                  I never returned.      1
552817  The opposite example would have been the Nativ...      1
936470  You are missing the point... The government sh...      1
550151              I guess every sarcastic post needs an      1
411843  I did not have sexual relations with that calc...      0
110866  Yes, but we are the only ones ever to set foot...      1
540740              Yeah &amp; that makes it okay, right?      1
999164  I didn't buy L4D2 coz I was too busy "boycotti...      0
958427        They still have cigarette lighters in cars.      0
781256  I agree, but I think it will be much harder fo...      0
633359  It's you dirty Royals fans... You say you don'...      1
984212        Well clearly, he wasn't a *true* Christian.      1
555647             ...tha

In [52]:
dataset = Dataset.from_pandas(df)
del df
gc.collect()


39

In [66]:
def get_training_corpus():
  training_corpus = (
    dataset["comment"][i : i + 1000]
    for i in range(0, len(dataset["comment"]), 1000)
  )
  return training_corpus

training_corpus = get_training_corpus()


In [ ]:
old_tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer = old_tokenizer.train_new_from_iterator(training_corpus, 52000)
tokenizer.save_pretrained("Sarcasm_Detection-Tokenizer")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [30]:
files.upload();

Saving Sarcasm_Detection-Tokenizer.zip to Sarcasm_Detection-Tokenizer.zip


In [35]:
!unzip '/content/Sarcasm_Detection-Tokenizer.zip' -d '/content/demos/Contributor_demos/Sarcasm Detection/'

Archive:  /content/Sarcasm_Detection-Tokenizer.zip
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/Sarcasm_Detection-Tokenizer/kaggle.json  
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/Sarcasm_Detection-Tokenizer/merges.txt  
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/Sarcasm_Detection-Tokenizer/special_tokens_map.json  
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/Sarcasm_Detection-Tokenizer/tokenizer.json  
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/Sarcasm_Detection-Tokenizer/tokenizer_config.json  
  inflating: /content/demos/Contributor_demos/Sarcasm Detection/Sarcasm_Detection-Tokenizer/vocab.json  


In [37]:
tokenizer = AutoTokenizer.from_pretrained("/content/demos/Contributor_demos/Sarcasm Detection/Sarcasm_Detection-Tokenizer")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
print(dataset["comment"][1], dataset["label"][1])
tokens = tokenizer(dataset["comment"][1:5], padding=True)
print(tokens)
decoded_string = tokenizer.decode(tokens.get("input_ids")[0])
print(decoded_string)

You do know west teams play against west teams more than east teams right? 0
{'input_ids': [[446, 363, 465, 5140, 2347, 528, 1095, 5140, 2347, 506, 525, 5338, 2347, 539, 31, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000], [741, 631, 42310, 4887, 1953, 12, 392, 1014, 18461, 312, 10837, 371, 15283, 12, 265, 9009, 1698, 515, 5277, 284, 14656, 742, 17], [677, 3195, 781, 320, 1460, 3407, 302, 265, 470, 9154, 25745, 12973, 2, 1671, 354, 14, 52000, 52000, 52000, 52000, 52000, 52000, 52000], [41, 609, 713, 479, 302, 646, 7910, 14, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}
You do know west teams play against west teams more

In [75]:
def tokenize_function(examples):
    return tokenizer(examples, padding="max_length", truncation=True)

def get_tokenized_dataset():
  tokenized_dataset = ([tokenize_function(token) for token in batch] for batch in training_corpus)
  return tokenized_dataset

tokenized_dataset = get_tokenized_dataset()

In [80]:
i:int = 0
for batch in tokenized_dataset:
  for token in batch:
    print(token)
    i+= 1
    if i > 3:
      break
  break

{'input_ids': [481, 312, 1709, 284, 408, 3985, 2611, 309, 503, 354, 367, 1039, 2887, 30481, 44410, 301, 24774, 365, 389, 265, 9019, 23320, 664, 1359, 7087, 1028, 14, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 52000, 5

In [38]:
model = AutoModel.from_pretrained("gpt2")

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

In [ ]:
from ivy.stateful.module import Module
from ivy.stateful.sequential import Sequential
from ivy.stateful.layers import *
from ivy.stateful.losses import *
from ivy.stateful.optimizers import *
from ivy.stateful.activations import *
from ivy.stateful.initializers import *
from ivy.stateful.norms import *